<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_CS6910/blob/main/Assignment2/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks : Training from scratch

# Initial setup

In [51]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv1D,Conv2D, BatchNormalization, Dropout, Activation, Softmax, MaxPooling2D
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam, RMSprop

import numpy as np
import pandas as pd
import os
import random
import pprint

from  matplotlib import pyplot as plt
%matplotlib inline

The dataset in our case was uploaded on google drive. To access the same, we have to mount drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

In [ ]:
!pip install wandb
!wandb login
import wandb
from wandb.keras import WandbCallback
wandb.init(project="CS6910-assg2", entity="swathi")

# Function to load data

In [54]:
def load_data(dir_train, dir_test, batch):
  
  seed = 42
  
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    samplewise_center = 0,
    horizontal_flip = True,
    rotation_range = 30,
    validation_split = 0.1)
  
  val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    samplewise_center = 0,
    validation_split = 0.1)
  
  test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    samplewise_center = 0)
  
  train_aug_dataset = train_datagen.flow_from_directory(
    dir_train,
    target_size = (img_dim,img_dim),
    batch_size = batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = 'training',
    seed = seed)

  train_dataset = val_datagen.flow_from_directory(
    dir_train,
    target_size = (img_dim,img_dim),
    batch_size = batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = 'training',
    seed = seed)
  
  val_dataset = val_datagen.flow_from_directory(
    dir_train,
    target_size = (img_dim,img_dim),
    batch_size = batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = 'validation',
    seed = seed)
  
  test_dataset = test_datagen.flow_from_directory(
    dir_test,
    target_size = (img_dim,img_dim),
    batch_size = batch,
    classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia'],
    class_mode='categorical',
    subset = None,
    seed = seed
  )
  
  return train_aug_dataset, train_dataset, val_dataset, test_dataset

# Functions to define models

In [55]:
# Without Batch Normalization
def CNN_wo_BN(filter_matrix, kernel_matrix, activation_matrix, fdropout, dense_no, learning_rate):
  # Input layer
  X_input = keras.Input(shape=(img_dim,img_dim,channel_no))
  # Layer 1
  X = Conv2D(filter_matrix[0], kernel_matrix[0])(X_input)
  X = Activation(activation_matrix[0])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  # Layer 2
  X = Conv2D(filter_matrix[1], kernel_matrix[1])(X)
  X = Activation(activation_matrix[1])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  # Layer 3
  X = Conv2D(filter_matrix[2], kernel_matrix[2])(X)
  X = Activation(activation_matrix[2])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  # Layer 4
  X = Conv2D(filter_matrix[3], kernel_matrix[3])(X)
  X = Activation(activation_matrix[3])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  # Layer 5
  X = Conv2D(filter_matrix[4], kernel_matrix[4])(X)
  X = Activation(activation_matrix[4])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  # Dense layers
  X = Flatten()(X)
  X = Dense(dense_no,activation=activation_matrix[5])(X)
  X = Dropout(fdropout)(X)
  X = Dense(10,activation='softmax')(X)
  model=Model(inputs=X_input,outputs=X)
  model.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy',  metrics=['accuracy'])
  return model

# With Batch Normalization
def CNN_w_BN(filter_matrix, kernel_matrix, activation_matrix, fdropout, dense_no, learning_rate):
  # Input layer
  X_input = keras.Input(shape=(img_dim,img_dim,channel_no))
  # Layer 1
  X = Conv2D(filter_matrix[0], kernel_matrix[0])(X_input)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[0])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  # Layer 2
  X = Conv2D(filter_matrix[1], kernel_matrix[1])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[1])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  # Layer 3
  X = Conv2D(filter_matrix[2], kernel_matrix[2])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[2])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  # Layer 4
  X = Conv2D(filter_matrix[3], kernel_matrix[3])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[3])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  # Layer 5
  X = Conv2D(filter_matrix[4], kernel_matrix[4])(X)
  X = BatchNormalization()(X)
  X = Activation(activation_matrix[4])(X)
  X = MaxPooling2D(pool_size=(2, 2))(X)
  # Dense layers
  X = Flatten()(X)
  X = Dense(dense_no,activation=activation_matrix[5])(X)
  X = Dropout(fdropout)(X)
  X = Dense(10,activation='softmax')(X)
  model=Model(inputs=X_input,outputs=X)
  model.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy',  metrics=['accuracy'])
  return model


def model_choose(filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no, batch_norm, learning_rate):
  wandb.log({"Filters": filter_matrix})
  if batch_norm == True:
    model = CNN_w_BN(filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no, learning_rate)
  else:
    model = CNN_wo_BN(filter_matrix, kernel_matrix, activation_matrix, dropout, dense_no, learning_rate)
  return model

# Defining parameters

In [56]:
global img_dim
img_dim = 200

global channel_no
channel_no = 3
#3 for RGB images, 1 for greyscale

global batch
batch = 128

global filters
filters = [[64,128,256,512,512],[64,128,256,256,512],[128,256,256,512,512],[96,128,256,512,512]]

global kernels
kernels = [[(3,3)]*5, [(5,5)]*5]

global activations
activations = [['relu']*6]

global F, K, A

global train_aug_dataset, train_dataset, val_dataset, test_datagen
train_aug_dataset, train_dataset, val_dataset, test_dataset = load_data('/content/drive/MyDrive/inaturalist_12K/train', '/content/drive/MyDrive/inaturalist_12K/val', batch)

# Hyperparameter sweeps

In [ ]:
sweep_config = {
    'method': 'grid'
    }

parameters_dict = {
    'filter_matrix':{
        'values': [3] # 0,1,2,3
      },
    'kernel_matrix': {
        'values': [1] # 0,1
      },
    'activation_matrix': {
        'values': [0]
      },
    'learn_rate': {
        'values': [1e-3] # 1e-3, 0.3
      },
    'epochs': {
        'values': [40]
      },
    'dropout': {
        'values': [0.5] # 0.2, 0.5
      },
    'batch_normalization': {
        'values': [False] # True, False
      },
    'dense_number': {
          'values': [512]
      },
    'augmentation':{
        'values' : [True] # True, False
      }
}
sweep_config['parameters'] = parameters_dict
pprint.pprint(sweep_config)

def training_sweep(config=None):
    with wandb.init(config=config):
        config = wandb.config

        F = filters[config.filter_matrix]
        K = kernels[config.kernel_matrix]
        A = activations[config.activation_matrix]

        model = model_choose(F, K, A, config.dropout, config.dense_number, config.batch_normalization, config.learn_rate)
        if config.augmentation == True:
          train = train_aug_dataset
        else:
          train = train_dataset

        history = model.fit(train, 
                            epochs=config.epochs,
                            validation_data = val_dataset,
                            callbacks = [WandbCallback(data_type='image', labels = classes)]
                            )
        ### On the best model only: 
        # model.save('/content/drive/MyDrive/Assgn2model')

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="CS6910-assg2")
wandb.agent(sweep_id, training_sweep)

# Results on test data

To get the accuracy on the test dataset:

In [ ]:
model = keras.models.load_model('/content/drive/MyDrive/Assgn2model')
model.summary()
result = model.evaluate(test_dataset)

Two ImageGenerator iterators are taken to plot random images, instead of the entire test dataset

In [59]:
(x,y) = test_dataset.next()
(x2,y2) = test_dataset.next()
predictions1 = model.predict(x)
predictions2 = model.predict(x2)

In [ ]:
fig, a = plt.subplots(3, 10, figsize=(30,30))
for i in range(0,3):
  for j in range(0,10):
    rand = random.randint(0,batch-1)
    if j%2 == 0:
      a[i][j].imshow(x[rand])
      one_hot=y[rand]
      pred = np.argmax(predictions[rand])
    else:
      a[i][j].imshow(x2[rand])
      one_hot=y2[rand]
      pred = np.argmax(predictions2[rand])
    for k in range(0,10):
      if one_hot[k]==1:
        name = classes[k]
    title = name + "(predicted:" + classes[pred] + ")"
    if name == classes[pred]:
      col = 'g'
    else:
      col = 'r'
    a[i][j].set_title(title, color = col)
fig.tight_layout(pad = 0, h_pad = -100)
plt.savefig('/content/drive/MyDrive/Best_model_output.png')
plt.show()

# Visualising filters

Creating a new model upto the first layer, and predicting outputs on one random image from test dataset. These outputs are the filters.

In [ ]:
NewModel = Model(inputs = [model.inputs],outputs = [model.get_layer(index=1).output])
NewModel.summary()

num = random.randint(0,127)
input = np.expand_dims(x[num], axis = 0)
outputs = NewModel.predict(input)

To see the original image:

In [ ]:
plt.imshow(x[num])

To view the filters as a grid:

In [ ]:
fig, a = plt.subplots(12, 8, figsize=(20,20))
for i in range(0,12):
  for j in range(0,8):
    a[i][j].imshow(outputs[0,:,:,12*j+i], cmap='gray')
fig.tight_layout(pad = 0, h_pad =0)
plt.savefig('/content/drive/MyDrive/A_4_c.png')
plt.show()

# Guided Backpropagation

For a random input from test dataset

In [12]:
(x,y) = test_dataset.next()
num = random.randint(0,127)
x_backprop = np.expand_dims(x[num],axis=0)

On the best model, and layer index 13 (5th convolutional layer):

In [15]:
@tf.custom_gradient
def GBRelu(x):
  def gradient(dy):
    grad = tf.cast(dy>0,"float32")*tf.cast(x>0, "float32")*dy
    return  grad
  return tf.nn.relu(x), gradient

GBModel = Model(inputs = [model.inputs],outputs = [model.get_layer(index=13).output])
layer_dict = [layer for layer in GBModel.layers[1:] if hasattr(layer,'activation')]
for layer in layer_dict:
  if layer.activation == tf.keras.activations.relu:
    layer.activation = GBRelu

In [ ]:
grad_list=[]
for i in range(0,10):
  with tf.GradientTape(persistent=True) as tape:
    inputs = tf.Variable(tf.cast(x_backprop, tf.float32))
    outputs = GBModel(inputs)
    new = np.zeros((1,8,8,512))
    rand = np.random.randint(0,511)
    new[:,:,:,rand] = np.ones((1,8,8))
    outputs = outputs*new
  grads = tape.gradient(outputs,inputs)[0]
  grad_list.append(grads)
  del grads

Visualizing the original image

In [ ]:
plt.imshow(x[num])

Plotting the result of guided backpropagation for ten random neurons

In [ ]:
fig, a = plt.subplots(1, 10, figsize=(30,30))
for i in range(0,10):
  a[i].imshow(np.array(grad_list[i])*255)
plt.savefig ('/content/drive/MyDrive/GuidedBackprop.png')
plt.show()